# 1. Выбор датасета

In [1]:
from sklearn.datasets import load_wine

X, y = load_wine(return_X_y=True, as_frame=True)
classes = set(y)

classes

{0, 1, 2}

In [17]:
import pandas as pd 

data = load_wine(as_frame=True)

df = pd.DataFrame(data=data['data'], columns = data['feature_names'])
df.to_csv('dataset.csv', sep = ',', index = False)

# 2. Формирование выборок данных

In [2]:
df = X
df['y_true'] = y
bts = []
for i in range(3):
  df_bt = df.sample(frac=0.5)
  y_bt = df_bt['y_true']
  X_bt = df_bt.drop(columns=['y_true'])
  bts.append([X_bt, y_bt])

# 3. Выбор классификаторов

In [3]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC

models = [MultinomialNB(), KNeighborsClassifier(), LinearSVC()]

# 4. Обучение классификаторов

In [4]:
import pandas as pd
from sklearn.metrics import accuracy_score

preds = []
for i, model in enumerate(models):
  X_bt = bts[i][0]
  y_bt = bts[i][1]
  model.fit(X_bt, y_bt)
  y_pred = model.predict(X_bt)
  pred_df = pd.DataFrame(y_pred, index=X_bt.index)
  preds.append(pred_df)
  print(accuracy_score(y_bt, y_pred))



0.8651685393258427
0.8202247191011236
0.8539325842696629


/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


# 5. Объединение датасетов с предсказаниями

In [5]:
preds_df = pd.concat(preds, axis=1)
preds_df.head()

,0,0,0
0,0.0,0.0,NaN
1,0.0,NaN,NaN
2,NaN,0.0,0.0
3,0.0,0.0,NaN
4,1.0,0.0,NaN


In [19]:
preds_df.to_csv('labels.csv', sep = ',', index = True)

# 6. Формирование онтологии

In [6]:
!pip install owlready2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 25.5 MB 121.1 MB/s 
  Created wheel for owlready2: filename=Owlready2-0.39-cp38-cp38-linux_x86_64.whl size=22260931 sha256=1b5d964a5d8f0524a1ac58ac05d06965f60de4ef5cc2d3d6b5430a7d76144417
  Stored in directory: /root/.cache/pip/wheels/bd/16/5f/a0bfc34a8f7682cbd6b4d9cb1436c0a0a04ac3579394d7e28a
Successfully built owlready2


In [7]:
from owlready2 import *
onto = get_ontology('onto.owl')

In [8]:
import types

with onto:
  tsr = types.new_class('tsr', (ObjectProperty, TransitiveProperty, SymmetricProperty, ReflexiveProperty))
  ts = types.new_class('ts', (ObjectProperty, TransitiveProperty, SymmetricProperty))

  for cl in classes:
    onto_cl = types.new_class(f'Class{cl}', (Thing,))
    onto_cl_know = types.new_class(f'Know_Class{cl}', (Thing,))
    onto_cl_know.equivalent_to.append(tsr.only(onto_cl))
    onto_cl_believe = types.new_class(f'Believe_Class{cl}', (Thing,))
    onto_cl_believe.equivalent_to.append(ts.only(onto_cl))

onto.save('onto.owl', format='ntriples')



# 7. Создание индивидов для предсказаний
# 8. Установление неразличимости между предсказаниями для одного и того же объекта

In [9]:
with onto:
  for index, row in preds_df.iterrows():
    inst_old = None
    for j, pred in enumerate(preds_df.columns):
      cl = row.iloc[j]
      if not pd.isna(cl):
        cl = int(cl)
        onto_cl = onto[f'Class{cl}']
        inst_new = onto_cl(f'inst{index}_{j}')
        if inst_old != None:
          inst_old.tsr.append(inst_new)
          inst_old.ts.append(inst_new)
        inst_old = inst_new

onto.save('onto.owl', format='ntriples')


# 9. Ризонер для вывода отношений

In [10]:
with onto:
  sync_reasoner_pellet(infer_property_values=True, debug=2)

onto.save('reasoned.owl', format='ntriples')

* Owlready2 * Running Pellet...
    java -Xmx2000M -cp /usr/local/lib/python3.8/dist-packages/owlready2/pellet/antlr-runtime-3.2.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jena-tdb-0.10.0.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jena-iri-0.9.5.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/httpcore-4.2.2.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/aterm-java-1.6.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/pellet-2.3.1.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/owlapi-distribution-3.4.3-bin.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/log4j-1.2.16.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/xercesImpl-2.10.0.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jena-core-2.10.0.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/slf4j-log4j12-1.6.4.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/commons-codec-1.6.jar:/usr/local/lib/py

* Owlready * Adding relation onto.inst95_1 tsr onto.inst95_1
* Owlready * Adding relation onto.inst43_2 tsr onto.inst43_2
* Owlready * Adding relation onto.inst43_2 ts onto.inst43_2
* Owlready * Adding relation onto.inst134_1 tsr onto.inst134_1
* Owlready * Adding relation onto.inst134_1 ts onto.inst134_1
* Owlready * Adding relation onto.inst73_1 tsr onto.inst73_1
* Owlready * Adding relation onto.inst73_1 ts onto.inst73_1
* Owlready * Adding relation onto.inst23_2 tsr onto.inst23_2
* Owlready * Adding relation onto.inst23_2 tsr onto.inst23_0
* Owlready * Adding relation onto.inst23_2 ts onto.inst23_2
* Owlready * Adding relation onto.inst23_2 ts onto.inst23_0
* Owlready * Adding relation onto.inst129_1 tsr onto.inst129_1
* Owlready * Adding relation onto.inst129_1 ts onto.inst129_1
* Owlready * Adding relation onto.inst97_1 tsr onto.inst97_1
* Owlready * Adding relation onto.inst97_1 ts onto.inst97_1
* Owlready * Adding relation onto.inst106_1 tsr onto.inst106_1
* Owlready * Adding r

# 10. Перевод онтологии в CWA

In [11]:
with onto:
  for inst in onto.individuals():
    close_world(inst)

onto.save('reasoned_cwa.owl', format='ntriples')

# 11. Ризонер для классификации

In [12]:
with onto:
  sync_reasoner_pellet(debug=2)

onto.save('reasoned_cwa.owl', format='ntriples')

* Owlready2 * Running Pellet...
    java -Xmx2000M -cp /usr/local/lib/python3.8/dist-packages/owlready2/pellet/antlr-runtime-3.2.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jena-tdb-0.10.0.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jena-iri-0.9.5.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/httpcore-4.2.2.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/aterm-java-1.6.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/pellet-2.3.1.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/owlapi-distribution-3.4.3-bin.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/log4j-1.2.16.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/xercesImpl-2.10.0.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jena-core-2.10.0.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/slf4j-log4j12-1.6.4.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/commons-codec-1.6.jar:/usr/local/lib/py